In [2]:
import pandas as pd
import numpy as np
import os

data_path = '../../Data/Used Cars/'

bmw = pd.read_csv(data_path+'bmw.csv')
toyota = pd.read_csv(data_path+'toyota.csv')
audi = pd.read_csv(data_path+'audi.csv')

In [3]:
toyota.head(10)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0
5,GT86,2017,15998,Manual,26919,Petrol,260,36.2,2.0
6,GT86,2017,18522,Manual,10456,Petrol,145,36.2,2.0
7,GT86,2017,18995,Manual,12340,Petrol,145,36.2,2.0
8,GT86,2020,27998,Manual,516,Petrol,150,33.2,2.0
9,GT86,2016,13990,Manual,37999,Petrol,265,36.2,2.0


In [5]:
# Supongamos que X, es 'mpg':
X = toyota['mpg']
X.head(3)

0    36.2
1    36.2
2    36.2
Name: mpg, dtype: float64

In [6]:
# Y sería 'fuelType':
y = toyota['fuelType']
y.head(3)

0    Petrol
1    Petrol
2    Petrol
Name: fuelType, dtype: object

In [7]:
# Queremos partir X, en k clases, que en este caso:
k = len(y.unique())
print(k)
y.unique()

4


array(['Petrol', 'Other', 'Hybrid', 'Diesel'], dtype=object)

In [8]:
toyota.sample(1)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
2252,Yaris,2018,13495,Automatic,15750,Petrol,145,47.1,1.5


In [9]:
y.value_counts(True, dropna=False)

Petrol    0.606560
Hybrid    0.303206
Diesel    0.074651
Other     0.015583
Name: fuelType, dtype: float64

In [10]:
toyota.sample(1)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
2098,Yaris,2016,12500,Automatic,20216,Hybrid,0,78.0,1.5


In [11]:
toyota['transmission'].value_counts(True, dropna=False)

Manual       0.567824
Automatic    0.394331
Semi-Auto    0.037697
Other        0.000148
Name: transmission, dtype: float64

In [12]:
toyota['engineSize'].value_counts(1, dropna=False)

1.0    0.305580
1.5    0.211190
1.8    0.191303
1.3    0.089196
2.0    0.059068
1.2    0.044969
2.5    0.036955
1.6    0.025082
2.4    0.010389
2.2    0.008608
1.4    0.005936
2.8    0.005343
3.0    0.005194
0.0    0.000890
4.2    0.000148
4.5    0.000148
Name: engineSize, dtype: float64

In [13]:
toyota['engineSize'].value_counts(dropna=False)

1.0    2059
1.5    1423
1.8    1289
1.3     601
2.0     398
1.2     303
2.5     249
1.6     169
2.4      70
2.2      58
1.4      40
2.8      36
3.0      35
0.0       6
4.2       1
4.5       1
Name: engineSize, dtype: int64

# Análisis Discriminante

In [17]:
toyota.loc[toyota['fuelType']=='Petrol']['mpg'].mean()

58.0653290922437

In [18]:
toyota.loc[toyota['fuelType']=='Other']['mpg'].mean()

71.1847619047619

In [19]:
toyota.loc[toyota['fuelType']=='Diesel']['mpg'].mean()

52.06679920477137

In [20]:
toyota.loc[toyota['fuelType']=='Hybrid']['mpg'].mean()

75.28218306412138

In [22]:
toyota.sample(1)[['price', 'mpg', 'mileage']]

,price,mpg,mileage
1060,7298,55.0,32358


# Regresión Cresta Kernel

In [31]:
toyota.head(1)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0


In [32]:
toyota['car_age'] = 2022 - toyota['year']

## Definición del espacio X,y

In [36]:
X = toyota[['mileage', 'car_age']]
y = toyota['price']
toyota[['mileage', 'car_age', 'price']].head(3)

,mileage,car_age,price
0,24089,6,16000
1,18615,5,15995
2,27469,7,13998


$\Phi(x_i) = mileage_i + car\_age_i$

* Dónde: 
* x_i = (mileage_i, car\_age_i)


In [37]:
toyota['tr1'] = toyota['mileage'] + toyota['car_age']
toyota[['tr1', 'price']].head(3)

,tr1,price
0,24095,16000
1,18620,15995
2,27476,13998


# Pasos:

* 1. Definimos nuestro kernel de mapeo, es decir $\Phi(X)$.
* 2. Optimizamos la partición del nuevo espacio que es $\Phi(X)$, para máximizar la separación de los puntos. 
* 3. Obtenemos w, que es esa partición óptima del espacio $\Phi(X)$, esto lo hacemos a través de comparar el comportamiento de y (función de costo) dados los eigen vectores de cada una de las opciones de partición del plano. Seleccionando los eigenvectores que minimicen la función de costo. 
* 4. Creamos el hiperplano w, definido por esos eigenvectores. 
* 5. Definimos un mapeo para las nuevas observaciones, tal que considere el hiperplano W. De esta forma, la función de estimación de $\hat{y}$, sería la siguiente:
$$\hat{y}=w\Phi(X)$$

In [56]:
from sklearn.kernel_ridge import KernelRidge

krr = KernelRidge(alpha=10)
krr.fit(X,y)

KernelRidge(alpha=10)

In [57]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

NOTA: Las siguientes métricas se calculan a través de los residuales para modelos de regresión, su definición es la siguiente:

* Mean absolute error: MAE = $\frac{1}{n} \sum^{n}_{i=1} |y_i - \hat{y_i}|$
* Mean squared error: MSE = $\frac{1}{n} \sum^{n}_{i=1} (y_i - \hat{y_i})^{2}$

In [58]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y, krr.predict(X))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y, krr.predict(X))}')

Tu modelo tiene un error absoluto promedio de: 6410.4146134386365
Tu modelo tiene un error cuadrado promedio de: 82321435.2004367


In [52]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y, krr.predict(X))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y, krr.predict(X))}')

Tu modelo tiene un error absoluto promedio de: 6410.484635663506
Tu modelo tiene un error cuadrado promedio de: 82321402.01714793


In [45]:
y.describe()

count     6738.000000
mean     12522.391066
std       6345.017587
min        850.000000
25%       8290.000000
50%      10795.000000
75%      14995.000000
max      59995.000000
Name: price, dtype: float64

In [46]:
toyota['car_age'].describe()

count    6738.000000
mean        5.251855
std         2.204062
min         2.000000
25%         4.000000
50%         5.000000
75%         6.000000
max        24.000000
Name: car_age, dtype: float64

In [47]:
from sklearn.kernel_ridge import KernelRidge
x_aux = toyota[['price', 'mileage']]
y_aux = toyota['car_age']
krr = KernelRidge(alpha=.8)
krr.fit(x_aux, y_aux)

KernelRidge(alpha=0.8)

In [49]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y_aux, krr.predict(x_aux))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y_aux, krr.predict(x_aux))}')

Tu modelo tiene un error absoluto promedio de: 1.6020932443296387
Tu modelo tiene un error cuadrado promedio de: 4.46164467262921


# SGD

In [61]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X = toyota[['mileage', 'car_age']].copy()
y = toyota['price'].copy()

pipe = make_pipeline(StandardScaler(), SGDRegressor(max_iter=1000, tol=.001), verbose=2)

pipe.fit(X,y)

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ...... (step 2 of 2) Processing sgdregressor, total=   0.0s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdregressor', SGDRegressor())],
         verbose=2)

In [62]:
pipe.predict(X)

array([11509.14821659, 12807.7679259 , 10206.95741396, ...,
        6314.69496021,  5031.80920832,  5005.2528258 ])

In [63]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y, pipe.predict(X))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y, pipe.predict(X))}')

Tu modelo tiene un error absoluto promedio de: 4438.394051743081
Tu modelo tiene un error cuadrado promedio de: 33153859.859720647


In [52]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y, krr.predict(X))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y, krr.predict(X))}')

Tu modelo tiene un error absoluto promedio de: 6410.484635663506
Tu modelo tiene un error cuadrado promedio de: 82321402.01714793


# Regresión Bayesiana

In [92]:
from sklearn.linear_model import BayesianRidge

br = make_pipeline(StandardScaler(), BayesianRidge(), verbose=2)
br.fit(X,y)

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing bayesianridge, total=   0.0s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('bayesianridge', BayesianRidge())],
         verbose=2)

In [93]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y, br.predict(X))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y, br.predict(X))}')

Tu modelo tiene un error absoluto promedio de: 4417.623141247626
Tu modelo tiene un error cuadrado promedio de: 33114084.393356156


# LDA

In [65]:
toyota.head(2)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age,tr1
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0,6,24095
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0,5,18620


In [80]:
yd.value_counts(1, dropna=False, ascending=True).values

Other     0.015583
Diesel    0.074651
Hybrid    0.303206
Petrol    0.606560
Name: fuelType, dtype: float64

In [70]:
LinearDiscriminantAnalysis?

In [85]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
yd = toyota['fuelType'].copy()
Xd = toyota[['mpg', 'price', 'mileage', 'car_age']].copy()
#priors = yd.value_counts(1, dropna=False, ascending=True).values
lda = LinearDiscriminantAnalysis()
lda.fit(Xd, yd)
lda

LinearDiscriminantAnalysis()

In [86]:
lda.predict(Xd)

array(['Petrol', 'Petrol', 'Petrol', ..., 'Petrol', 'Petrol', 'Petrol'],
      dtype='<U6')

In [87]:
lda.predict_proba(Xd)

array([[0.14045438, 0.01287572, 0.00411995, 0.84254994],
       [0.06681596, 0.00654589, 0.00301732, 0.92362083],
       [0.10276017, 0.00718913, 0.00304975, 0.88700095],
       ...,
       [0.04161058, 0.01382977, 0.00573785, 0.9388218 ],
       [0.12987998, 0.00662188, 0.00239399, 0.86110415],
       [0.03043297, 0.00408601, 0.00283161, 0.96264941]])

In [88]:
from sklearn.metrics import accuracy_score

In [89]:
accuracy_score(yd, lda.predict(Xd))

0.8950727218759276

* Accuracy Score = $\frac{TN + TP}{TN + FP + TP + FN}$
* En otras palabras es...
* Accuracy Score = $\frac{Aciertos}{Total de observaciones}$

# Nota adicional: Train test split

In [95]:
#Clase que vive en el módulo:
from sklearn.model_selection import train_test_split

#Función:
'''Esta clase es utilizada para partir los datos en dos conjuntos, y poder darle a los modelos un espacio
    de entrenamiento y un segundo espacio de prueba o validación con datos que no han visto, de esta forma
    podemos evaluar que los modelos no se hayan aprendido los datos de memoria, si no que los modelos se 
    hayan aprendido la relación entre los datos.'''

#El uso es el siguiente:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3, random_state=42)

br.fit(X_train, y_train)

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing bayesianridge, total=   0.0s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('bayesianridge', BayesianRidge())],
         verbose=2)

In [97]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y_train, br.predict(X_train))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y_train, br.predict(X_train))}')

Tu modelo tiene un error absoluto promedio de: 4409.751083635469
Tu modelo tiene un error cuadrado promedio de: 33557829.71325712


In [96]:
print(f'Tu modelo tiene un error absoluto promedio de: {mean_absolute_error(y_test, br.predict(X_test))}')
print(f'Tu modelo tiene un error cuadrado promedio de: {mean_squared_error(y_test, br.predict(X_test))}')

Tu modelo tiene un error absoluto promedio de: 4378.965366114112
Tu modelo tiene un error cuadrado promedio de: 32099624.93527039


In [99]:
np.abs(33557829.71325712-32099624.93527039)/33557829.71325712

0.043453488811603914

# Persistencia del pipeline:

In [104]:
from joblib import dump, load
#Guardamos con dump:
dump(br, 'prueba0.joblib')
#Cargamos con load
model = load('prueba0.joblib')

In [103]:
model.predict(X_test)

array([15111.14850542,  5689.41533821, 11551.71825761, ...,
       12737.77623039,  7971.70622995, 15165.70123369])